In [1]:
1+1

2

In [2]:
# increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
# !pip install --quiet tensorflow-hub
# !pip install --quiet seaborn

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

W0401 15:43:00.600650 140262053664512 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import random
import time
import pickle
import gc
import re
import itertools
from importlib import reload

import scipy.sparse

In [8]:
from helpers import load_data
from helpers import explore_data
from helpers import preprocess_data
from helpers import train_model

In [9]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [10]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

Instructions for updating:
Colocations handled automatically by placer.


W0401 15:44:10.311100 140262053664512 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [8]:
type(embed)

tensorflow_hub.module.Module

In [11]:
main_dir = '/tf/notebooks/'

In [12]:
data_path = main_dir + 'final-project/data/sample/hein-daily'

In [13]:
all_speech_ids, all_speeches = load_data.load_speech_data(data_path)


File speeches_097.txt has 249718612 characters
and 283399 speeches

Speeches list has 283399 speeches


In [14]:
reload(load_data)
start_time = time.time()
descr = load_data.load_descr_data(main_dir+'final-project/data/QA/full_descr.txt')
print("\nIt took {0:.1f} seconds to create the dictionary".format(time.time()-start_time))

2914465 lines have been read
37617 keys had duplicates and deleted
The dictionary has 2808050 keys

Random congressperson: 1090105359
Party D
Chamber H
Ethnicity W
State NY
Age 77.0
speakerid 109121410.0
speaker Ms. SLAUGHTER
Full Name SLAUGHTER, LOUISE
NonWhite 0.0
Age_lt_med 0.0
date 20060201
match exact
word_count 12
check 1
Female 1.0
Congress 109
char_count 59

It took 16.8 seconds to create the dictionary


In [15]:
gender = load_data.create_target_labels(all_speech_ids, descr)[0]
len(gender)

283399

In [16]:
train, train_ids, train_target, val, val_ids, val_target, test, test_ids, test_target = preprocess_data.split_train_val_test(all_speeches, all_speech_ids, gender, descr)

Training split: 1622 ones and 1622 zeroes
Training speech list size: 3244
Training target list size: 3244
Validation split: 540 ones and 540 zeroes
Validation speech list size: 1080
Validation target list size: 1080
Test split: 542 ones and 542 zeroes
Test speech list size: 1084
Test target list size: 1084


In [17]:
chunk_len=30

In [18]:
reload(preprocess_data)
train_chunk, train_ids_chunk, train_target_chunk = preprocess_data.split_speech_to_chunks(train, train_ids, train_target, max_len=chunk_len)

Original data has 3244 speeches
It was split into 37989 chunks
Checks on ids and target 37989 37989
Original target mean 0.5
New target mean 0.5066203374661086


In [19]:
val_chunk, val_ids_chunk, val_target_chunk = preprocess_data.split_speech_to_chunks(val, val_ids, val_target, max_len=chunk_len)

Original data has 1080 speeches
It was split into 12903 chunks
Checks on ids and target 12903 12903
Original target mean 0.5
New target mean 0.4755483220956367


In [20]:
train_chunk[:2]

['Mr. Speaker. I want to compliment the gentleman from California for his comments. I think one of the things that he and the gentleman from Wisconsin were pointing out is',
 'that they had some perfecting punendments to the Commission that would have made it better. I think some of the things that people worried about were the calendar days rather']

In [21]:
# del descr

In [22]:
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

start_time = time.time()
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    train_embeddings = session.run(embed(train_chunk))
print("\nIt took {0:.1f} seconds to create embeddings".format(time.time()-start_time))


It took 40.5 seconds to create embeddings


In [23]:
train_embeddings.shape

(37989, 512)

In [24]:
start_time = time.time()
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    val_embeddings = session.run(embed(val_chunk))
print("\nIt took {0:.1f} seconds to create embeddings".format(time.time()-start_time))


It took 19.4 seconds to create embeddings


In [25]:
val_embeddings.shape

(12903, 512)

In [26]:
outdata_path = main_dir + 'final-project/Classification/data'

In [27]:
with open(os.path.join(outdata_path, 'train_emb30'), 'wb') as fp:
    pickle.dump(train_embeddings, fp)
with open(os.path.join(outdata_path, 'val_emb30'), 'wb') as fp:
    pickle.dump(val_embeddings, fp)

In [ ]:
with open(os.path.join(outdata_path, 'train_emb30'), 'rb') as fp:
    train_embeddings = pickle.load(fp)
with open(os.path.join(outdata_path, 'val_emb30'), 'rb') as fp:
    val_embeddings = pickle.load(fp)

In [29]:
np.array(train_embeddings).shape

(37989, 512)

In [242]:
def reshape_chunked_speech(data_emb, ids, ids_chunk, target, max_chunks=25):
    
    id_dict = {k:[] for k in ids} 
    for i in range(len(ids_chunk)):
        id_dict[ids_chunk[i]].append(data_emb[i]) 
        
    n_chunks = max_chunks
    data_list, new_ids = [], []
    for speech_id in id_dict:
        new_ids.append(speech_id)
        chunks = id_dict[speech_id].copy()
        if len(chunks) < n_chunks:
            padding = (n_chunks - len(id_dict[speech_id])) * [[0]*512]
            chunks.extend(padding)
            data_list.append(chunks)
        else:
            data_list.append(chunks[:n_chunks])    
            
    data_list_flat = list(itertools.chain.from_iterable(data_list))
    data_array = np.reshape(np.array(data_list_flat), [len(ids), -1, 512])
    print("Input data shape:",data_array.shape)
    
    new_target = [target[ids.index(new_ids[i])] for i in range(len(ids))]
    print("Target shape", len(new_target))
    
    return data_array, new_target, new_ids

In [243]:
train_emb, train_target_emb, train_ids_emb = reshape_chunked_speech(train_embeddings, train_ids, train_ids_chunk, train_target)

Input data shape: (3244, 25, 512)
Target shape 3244


In [244]:
val_emb, val_target_emb, val_ids_emb = reshape_chunked_speech(val_embeddings, val_ids, val_ids_chunk, val_target)

Input data shape: (1080, 25, 512)
Target shape 1080


In [245]:
logs_base_dir = main_dir+"final-project/Classification/logs"

In [246]:
cnn_model_params = {
    'model': 'cnn',
    'learning_rate': 0.001,
    'layers': 2,
    'epochs': 1000,
    'batch_size': 128,
    'filters': 64,
    'dropout_rate': 0.2,
    'kernel_size': 3,
    'pool_size': 1
}

In [251]:
len(train_emb)

3244

In [253]:
reload(train_model)
history, train_pred_probs, val_pred_probs = train_model.train_model(((train_emb, np.array(train_target_emb)), (val_emb, np.array(val_target_emb))), logs_base_dir, **cnn_model_params)

Train on 3244 samples, validate on 1080 samples
Epoch 1/1000
 - 2s - loss: 0.6812 - acc: 0.5629 - val_loss: 0.6538 - val_acc: 0.6287
Epoch 2/1000
 - 1s - loss: 0.6461 - acc: 0.6255 - val_loss: 0.6257 - val_acc: 0.6444
Epoch 3/1000
 - 1s - loss: 0.6255 - acc: 0.6489 - val_loss: 0.6174 - val_acc: 0.6648
Epoch 4/1000
 - 1s - loss: 0.6108 - acc: 0.6612 - val_loss: 0.6351 - val_acc: 0.6343
Epoch 5/1000
 - 1s - loss: 0.6108 - acc: 0.6732 - val_loss: 0.6061 - val_acc: 0.6685
Epoch 6/1000
 - 1s - loss: 0.5911 - acc: 0.6776 - val_loss: 0.6008 - val_acc: 0.6694
Epoch 7/1000
 - 1s - loss: 0.5770 - acc: 0.6899 - val_loss: 0.5956 - val_acc: 0.6731
Epoch 8/1000
 - 1s - loss: 0.5660 - acc: 0.7013 - val_loss: 0.6095 - val_acc: 0.6546
Epoch 9/1000
 - 1s - loss: 0.5623 - acc: 0.7053 - val_loss: 0.5975 - val_acc: 0.6750
Validation accuracy: 0.675000011920929, loss: 0.5975039340831615
